<a href="https://colab.research.google.com/github/ingabLee/Book_GenerateAIForComptureVision/blob/main/GenerateAI_Vision_Chapter_2_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install pytorch-model-summary

In [ ]:
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import pytorch_model_summary

from google.colab import drive

drive.mount('/content/drive')

dirWork = '/content/drive/MyDrive/VisionForGenerateAI/Chapter2'
dirImage = dirWork + '/VAE_img'

os.makedirs(dirImage, exist_ok=True)

Mounted at /content/drive


In [ ]:
# tensor data normalization. (find min, max value)
def normalization(tensor, min_value, max_value):
  min_tensor = tensor.min()
  tensor = (tensor - min_tensor)
  max_tensor = tensor.max()
  tensor = tensor / max_tensor
  tensor = tensor * (max_value - min_value) + min_value
  return tensor

# tensor value round
def value_round(tensor):
  return torch.round(tensor)

# 생성한 tensor을 이미지로 출력하기 위한 차원변환을 수행하는 함수.
def to_img(x):
  x = x.view(x.size(0), 1, 28, 28)
  return x

# MINST 데이터를 학습에 사용하기위해서 전처리를 수행하는 transform함수를 선언
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:value_round(tensor))
])

batch_size = 128    # 배치 사이즈 128

# load MNIST dataset with transform(전처리)
dataset = MNIST(dirWork + '/MNIST_dataset', transform=img_transform, download=True)

# 배치사이즈에 맞게 dataloader를 설정하여 shuffle옵션을 통해 섞는다.
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# 인코더, 디코더로 구성됀 변이형 오토인터코 선언.
class VariationalAutoencoder(nn.Module):
  def __init__(self):
    super(VariationalAutoencoder, self).__init__()

    # 인코더 생성. 두개의 선형 레이어와 한개의 활성화 함수레이어로 구성. 입력크기는
    # [N, 28*28]로 설정한다. N은 배치사이즈 이다..
    self.encoder = nn.Sequential(
        nn.Linear(28 * 28, 400),
        nn.ReLU(True),
        nn.Linear(400, 40))

    # 디코더 생성. 재매개변수화를 거쳐서 샘플링된 잠재벡터를 입력받기 때문에
    # 인코더 출력크기와 다른 입력 크기를 가진다. 각각 두개의 선형 레이어와 활성화 함수로 구성
    self.decoder = nn.Sequential(
        nn.Linear(20, 400),
        nn.ReLU(True),
        nn.Linear(400, 28*28),
        nn.Sigmoid())

  # 재매개변수화를 위한 함수정의. 재매개변수 함수내에서는 표준정규분포로부터 샘플링한 값을
  # 평균과 분산 입력을 통해 수동으로 변환하여 잠재벡터를 반환한다.
  def reparameterization(self, mean, lovar):
    var = logvar.exp()
    std = var.sqrt()
    eps = Variable(torch.cuda.FloatTensor(std.size()).normal_())

    return eps.mul(std).add(mean)

  # 변이형 오토 인코더의 동작순서를 정의. 인코더가 출력한 벡터의 절반은 평균, 나머지는 분산으로 활용하여
  # 재매개변수화를 통해 구성한 잠재벡터를 디코딩하여 이미지를 생성
  def forward(self, x):
    h = self.encoder(x)
    mean = h[ : , :20]
    logvar = h[ : , 20 : ]
    z = self.reparameterization(mean, logvar)
    x_gen = self.decoder(z)
    return x_gen, mean, logvar

  # 잠재공간내 두 잠재벡터의 선형보간법적용을 통해 생성된 이미지들의 경향성을 확인하기 위한 함수
  def interpolation(self, x_1, x_2, alpha):
    traverse_1 = self.encoder(x_1)
    traverse_2 = self.encoder(x_2)
    mean_1, mean_2 = traverse_1[:, :20], traverse_2[:, :20]
    logvar_1, logvar_2 = traverse_1[:,:20], traverse_2[:, 20:]
    traverse_m = (1-alpha) * mean_1 + alpha * mean_2
    traverse_logvar = (1-alpha) * logvar_1 * alpha * logvar_2
    z = self.reparameterization(traverse_m, traverse_logvar)
    generated_image = self.decoder(z)

    return generated_image



In [ ]:
# 변이형 오토 인코더를 생성하고 GPU에 모델 로드
model = VariationalAutoencoder()

#pytorch_model_summary 라이브러리를 통해 변이형 오토인코더의 구조를 확인.
# 입력 벡터는 (1, 784)의 크기를 가진 영벡터로 설정했고, 연산결과에 따른
# 모델구조를 출력
print(pytorch_model_summary.summary(model, torch_zeros(1, 784).cuda(),
                                    show_input=True))

In [ ]:
BCE = nn.BCELoss()
num_epochs, learning_rate = 50, 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  for data in dataloader:
    img, _ = data